In [1]:
from src.utils.wa_utils import (
    search_wa_athletes,                                     # WA autocomplete (fallback)
    fetch_and_store_wa_results                              # WA scraping (fallback)
)

from src.utils.scraping_wa import (
    search_athletes_by_name as _wa_search,
    get_athlete_results_by_name as _wa_results,
)

In [2]:
_wa_search('vaudois')

,aaAthleteId,familyName,givenName,birthDate,disciplines,iaafId,gender,country,urlSlug,__typename
0,14812063,VAUDOIS,Aurelien,06 MAR 1998,"4x800 Metres Relay, 800 Metres, 800 Metres Sho...",404944,Men,FRA,france/aurelien-vaudois-014812063,AthleteSearchResult


In [ ]:
from src.utils.http_utils import search_athletes            # FFA autocomplete

In [14]:
search_athletes("habz")

[{'hactseq': '7543814',
  'name': 'HABZ Azeddine',
  'club': '',
  'sex': 'M',
  'seq': '4455465347524851435650494752'}]

In [17]:
import requests
from bs4 import BeautifulSoup

# URL à scraper
url = "https://bases.athle.fr/asp.net/athletes.aspx?base=bilans&seq=4455465347524851435650494752"

# Envoyer la requête au site web
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)

# Vérifier si la requête a réussi
if response.status_code == 200:
    # Parser le contenu HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Chercher les cellules td avec style="width:40%"
    target_element = soup.find('td', style="width:40%")
    
    # Si l'élément est trouvé, extraire le texte à l'intérieur du b
    if target_element and target_element.find('b'):
        date_naissance = target_element.find('b').text.strip()
        print("Date de naissance:", date_naissance)
    else:
        print("Élément non trouvé. La structure de la page a peut-être changé.")
else:
    print(f"Échec de récupération de la page web. Code de statut: {response.status_code}")

Date de naissance: 19/07/1993


In [18]:
response.content

b'\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml" lang="fr" xml:lang="fr">\r\n\r\n<head>\r\n\t\r\n\t<title>les Bilans</title>\r\n\t\r\n\r\n<meta http-equiv="Content-Type" content="text/html;charset=utf-8" />\r\n<meta http-equiv="content-language" content="fr" />\r\n<meta name="description" content="Base de Donn\xc3\xa9es - F\xc3\xa9d\xc3\xa9ration Fran\xc3\xa7aise d\'Athl\xc3\xa9tisme" />\r\n<meta name="keywords" content="Base de Donn\xc3\xa9es - F\xc3\xa9d\xc3\xa9ration Fran\xc3\xa7aise d\'Athl\xc3\xa9tisme" />\r\n<meta name="robots" content="noindex, nofollow" />\r\n<script async src="https://www.googletagmanager.com/gtag/js?id=G-8XBNLRQE78"></script>\r\n<script>\r\n  window.dataLayer = window.dataLayer || [];\r\n  function gtag(){dataLayer.push(arguments);}\r\n  gtag(\'js\', new Date());\r\n\r\n  gtag(\'config\', \'G-8XBNLRQE78\');\r\n</script>\r\n<scri

In [22]:
import requests
import re

# URL to scrape
url = "https://bases.athle.fr/asp.net/athletes.aspx?base=bilans&seq=4851475245544851495050494554"

# Send request to the website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)

# Check if request was successful
if response.status_code == 200:
    # Use regex to directly extract the year from the HTML
    match = re.search(r'<td style="width:40%"><b>(\d{4})</b></td>', response.text)
    if match:
        birth_year = match.group(1)
        print("Birth year:", birth_year)
    else:
        print("Birth year not found")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Birth year: 1998


In [23]:
import requests
import re

# URL to scrape 4455465347524851435650494752
url = "https://bases.athle.fr/asp.net/athletes.aspx?base=bilans&seq=4455465347524851435650494752"

# Send request to the website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)

# Check if request was successful
if response.status_code == 200:
    # Use regex to directly extract date/year from the HTML - handles both "1998" and "19/07/1993" formats
    match = re.search(r'<td style="width:40%"><b>([^<]+)</b></td>', response.text)
    if match:
        birth_date = match.group(1)
        print("Birth date:", birth_date)
    else:
        print("Birth date not found")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Birth date: 19/07/1993


In [24]:
import requests
import re

def scrape_birth_date(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Use stream=True to stop downloading once we find our match
    with requests.get(url, headers=headers, stream=True) as response:
        if response.status_code != 200:
            return f"Failed to retrieve the webpage. Status code: {response.status_code}"
        
        # Initialize buffer and pattern
        chunk_size = 8192
        pattern = re.compile(r'<td style="width:40%"><b>([^<]+)</b></td>')
        buffer = ""
        
        # Process the response in chunks
        for chunk in response.iter_content(chunk_size=chunk_size, decode_unicode=True):
            if not chunk:
                continue
                
            # Add chunk to buffer and search for pattern
            buffer += chunk
            match = pattern.search(buffer)
            
            if match:
                return match.group(1)
                
            # Keep only the last 100 characters in case pattern spans chunks
            buffer = buffer[-100:] if len(buffer) > 100 else buffer
            
        return "Birth date not found"

# Example usage
url = "https://bases.athle.fr/asp.net/athletes.aspx?base=bilans&seq=4851475245544851495050494554"
birth_date = scrape_birth_date(url)
print("Birth date:", birth_date)

Birth date: 1998
